In [1]:
import graphlab as gl
import numpy as np
from pylab import *

This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489253713.log


In [2]:
def sig(t):
    sig = 1. / (1+np.exp(-t))
    return sig

In [ ]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [2]:
comb = gl.SFrame.read_csv('/Users/lizhu/Dropbox/projects/calcium/format4ML/format4ML_GC6f_emx_01_windowLen200_winStep_050_v2_threshold_10.csv', delimiter=',',header=False,verbose = False)
colName_dg = 'degree'
colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])

colName_cc = 'clusteringCoef'
colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])

colName_pl = 'pathlength'
colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])

colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
colName = (list(colName))
# type(colName)

dictionary = dict(zip(comb.column_names(), colName))
comb = comb.rename(dictionary)
comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)

# comb['Whisker'].show(view = 'Categorical')

In [6]:
# NOTE：run the functions below before running this section!!!

feature_dg = comb.column_names()[0:30] # feature 1~30: degree, 31~60: clustering coefficient, 60~90: pathlength
feature_cc = comb.column_names()[30:60]
feature_pl = comb.column_names()[60:90]

l2_penalty = 0.01
l1_penalty = 0

acc,SE,SP = lz_logistic_acc_SE_SP(comb, feature_dg, l2_penalty, l1_penalty)
# acc,SE,SP = lz_logistic_acc_SE_SP(comb, feature_dg+feature_pl+feature_cc, l2_penalty, l1_penalty)

acc_mean =  0.888498789346
acc_std =  0.0363610479115
SE_mean =  0.671912353205
SE_std =  0.103620159816
SP_mean =  0.946521169739
SP_std =  0.0245938233062


In [4]:
############### this results have been used for CLEO paper

# comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=2)

def lz_logistic_acc_SE_SP(data, feature, l2_penalty, l1_penalty):
    
    folds = gl.cross_validation.KFold(comb, 10)
    SE=[None]*10
    SP=[None]*10
    acc=[None]*10
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.logistic_classifier.create(train,
                                          target='Whisker',
                                          features=feature,
                                          l2_penalty = l2_penalty, 
                                          l1_penalty = l1_penalty,
                                          validation_set=None,
                                          verbose = False)
        confusion_matrix = m.evaluate(valid, 'confusion_matrix')
        confusion_matrix = confusion_matrix.values()[0]
        
        TP, TN, FP, FN = lz_extract_ACC_SE_SP(confusion_matrix)
        
        SP[idx] = float(TN) / (TN + FP)
        SE[idx] = float(TP) / (TP + FN)
        acc[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
    
    print 'acc_mean = ', np.mean(acc)
    print 'acc_std = ', np.std(acc)
    print 'SE_mean = ', np.mean(SE)
    print 'SE_std = ', np.std(SE)
    print 'SP_mean = ', np.mean(SP)
    print 'SP_std = ', np.std(SP)
    
    return acc, SE, SP

In [5]:
def lz_extract_ACC_SE_SP(confusion_matrix):
    TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
    if np.size(TP) == 0:
        TP = 0
    else:
        TP = TP['count'][0]
    TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
    if np.size(TN) == 0:
        TN = 0
    else:
        TN = TN['count'][0]
    FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
    if np.size(FP) == 0:
        FP = 0
    else:
        FP = FP['count'][0]
    FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
    if np.size(FN) == 0:
        FN = 0
    else:
        FN = FN['count'][0]
    
    return TP, TN, FP, FN

In [ ]:
############### this is the normalized version of the above code

# normalize the features
# feature_matrix is a SFrame, first 90 columns are features, last column is labels.
def normalize_features_mice(comb):
    # extract feature columns
    comb_np = comb.to_numpy()
    comb_feature = comb_np[:,0:90]
    
    # compute normed features and norms 
    norms = np.linalg.norm(comb_feature, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    comb_feature_normed = comb_feature / norms
    
    # combine normalized feature vectors and the labels
    comb_normed = np.concatenate((comb_feature_normed,comb_np[:,90:91]),axis = 1)
    # convert numpy array to SFrame
    comb_normed = gl.SFrame(comb_normed)
    comb_normed = comb_normed.unpack('X1', '')
    dictionary_norm = dict(zip(comb_normed.column_names(), colName))
    comb_normed = comb_normed.rename(dictionary_norm) # this is the normed version

    return comb_normed, norms # note: norms is array, comb_normed is SFrame

# comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=2)
def lz_logistic(data, feature):
    
    # normalize 
    comb, norms = normalize_features_mice(data)
    comb['Whisker'] = comb['Whisker'].astype(int)

    folds = gl.cross_validation.KFold(comb, 5)
    SE  = [None]*5
    SP  = [None]*5
    acc = [None]*5
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.logistic_classifier.create(train,
                                          target='Whisker',
                                          features=feature,
                                          l2_penalty = 0.05,
                                          validation_set=None,
                                          verbose = False)
        
        # normalize the validation dataset
        valid_np = valid.to_numpy()
        valid_feature = valid_np[:,0:90]

        # use normed from features
        valid_feature_normed = valid_feature / norms

        # combine normalized feature vectors and the labels
        valid_normed = np.concatenate((valid_feature_normed,valid_np[:,90:91]),axis = 1)

        # convert numpy array to SFrame
        valid_normed = gl.SFrame(valid_normed)
        valid_normed = valid_normed.unpack('X1', '')
        dictionary_norm = dict(zip(valid_normed.column_names(), colName))
        valid_normed = valid_normed.rename(dictionary_norm) # this is the normed version
        valid_normed['Whisker'] = valid_normed['Whisker'].astype(int)

        confusion_matrix = m.evaluate(valid, 'confusion_matrix')
        confusion_matrix = confusion_matrix.values()[0]
        TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
        if np.size(TP) == 0:
            TP = 0
        else:
            TP = TP['count'][0]
        TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
        if np.size(TN) == 0:
            TN = 0
        else:
            TN = TN['count'][0]
        FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
        if np.size(FP) == 0:
            FP = 0
        else:
            FP = FP['count'][0]
        FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
        if np.size(FN) == 0:
            FN = 0
        else:
            FN = FN['count'][0]
        SE[idx] = float(TN) / (TN + FP)
        SP[idx] = float(TP) / (TP + FN)
        acc[idx] = float(TP+TN) / (TP+TN+FP+FN)
        idx = idx + 1
    
    print 'acc_mean = ', np.mean(acc)
    print 'acc_std = ', np.std(acc)
    print 'SE_mean = ', np.mean(SE)
    print 'SE_std = ', np.std(SE)
    print 'SP_mean = ', np.mean(SP)
    print 'SP_std = ', np.std(SP)
    
    return acc, SE, SP

# normalize

feature = comb.column_names()[60:90] 

acc,SE,SP = lz_logistic(comb, feature)



In [ ]:
# exploring the data
num_wm = len(comb[comb['Whisker']==1])
num_wn = len(comb[comb['Whisker']==0])
# major class classifier(naive classifier)
major_classifier = float(num_wn)/(num_wm + num_wn)
major_classifier
num_wn

In [ ]:
train_data,test_data = comb.random_split(.8, seed=0)
# train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)
train_data = gl.toolkits.cross_validation.shuffle(train_data, random_seed=1)

In [ ]:
folds = gl.cross_validation.KFold(comb, 5)
folds
params = dict([('target', 'Whisker'), ('features', comb.column_names()[0:90]), ('validation_set', None)])

all_feature_model = gl.cross_validation.cross_val_score(folds,
                                              gl.logistic_classifier.create,
                                              params)

In [ ]:
# print all_feature_model.get_results()
all_feature_model.get_results()

In [ ]:
mylist = sum([range(0,30)] + [range(60,90)],[])
mylist[0:3,5]

In [ ]:
type(colName)

In [ ]:
whisker_model.evaluate(test_data)

In [ ]:
## Weight 
weight = whisker_model['coefficients'].sort('value', ascending=False)
weight['name']

In [ ]:
## 